### Ensemble Model(+Optuna)_Version2 for Sales Success Prediction

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import mean_squared_error


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import optuna
import plotly
from catboost import CatBoostClassifier

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("submission.csv") 

Data Processing

In [ ]:
def label_encoding(series: pd.Series) -> pd.Series:
  

    my_dict = {}


    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [ ]:
# Label Encoding
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])
    
df_all

,customer_country,business_subarea,business_area,business_unit,customer_type,enterprise,customer_job,inquiry_type,product_category,product_subcategory,product_modelname,customer_position,response_corporate,expected_timeline
0,9070,28,0,0,10,0,420,32,180,349,699,47,33,246
1,8406,0,0,0,10,0,303,32,180,349,699,21,33,246
2,6535,17,0,0,10,0,160,27,238,349,699,80,21,246
3,3388,44,0,0,10,0,166,32,318,349,699,21,21,246
4,5799,86,0,0,29,0,84,32,180,349,699,80,21,246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5266,10650,86,6,0,33,0,468,30,183,349,699,9,43,271
5267,16640,86,6,3,33,0,323,30,183,349,699,74,50,271
5268,1079,86,6,0,29,0,166,58,180,349,699,64,43,246
5269,24,86,6,3,9,1,323,58,52,349,699,74,12,271


In [6]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59299 entries, 0 to 59298
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bant_submit              59299 non-null  float64
 1   customer_country         59299 non-null  int64  
 2   business_unit            59299 non-null  int64  
 3   com_reg_ver_win_rate     14568 non-null  float64
 4   customer_idx             59299 non-null  int64  
 5   customer_type            59299 non-null  int64  
 6   enterprise               59299 non-null  int64  
 7   historical_existing_cnt  13756 non-null  float64
 8   id_strategic_ver         3444 non-null   float64
 9   it_strategic_ver         1121 non-null   float64
 10  idit_strategic_ver       4565 non-null   float64
 11  customer_job             59299 non-null  int64  
 12  lead_desc_length         59299 non-null  int64  
 13  inquiry_type             59299 non-null  int64  
 14  product_category      

In [8]:
df_train1 = df_train[df_train['is_converted'] == 1]
df_train2 = df_train[df_train['is_converted'] == 0]

df_train2 = df_train2.iloc[:25000]
#df_train2.info()

df_train = pd.concat([df_train1, df_train2])


df_train.loc[np.isnan(df_train.com_reg_ver_win_rate)==True,'com_reg_ver_win_rate'] = df_train.com_reg_ver_win_rate.median()
df_train.loc[np.isnan(df_train.historical_existing_cnt)==True,'historical_existing_cnt'] = df_train.historical_existing_cnt.median()
df_train.loc[np.isnan(df_train.id_strategic_ver)==True,'id_strategic_ver'] = df_train.id_strategic_ver.fillna(0)
df_train.loc[np.isnan(df_train.it_strategic_ver)==True,'it_strategic_ver'] = df_train.it_strategic_ver.fillna(0)
df_train.loc[np.isnan(df_train.idit_strategic_ver)==True,'idit_strategic_ver'] = df_train.idit_strategic_ver.fillna(0)
df_train.loc[np.isnan(df_train.ver_win_rate_x)==True,'ver_win_rate_x'] = df_train.ver_win_rate_x.median()
df_train.loc[np.isnan(df_train.ver_win_ratio_per_bu)==True,'ver_win_ratio_per_bu'] = df_train.ver_win_ratio_per_bu.median()

df_train = df_train.drop(['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'lead_desc_length',
                         'customer_type', 'historical_existing_cnt', 'product_subcategory', 'customer_country.1' ,
                          'product_modelname', 'expected_timeline', 'business_subarea'], axis=1)

df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29850 entries, 0 to 27624
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bant_submit           29850 non-null  float64
 1   customer_country      29850 non-null  int64  
 2   business_unit         29850 non-null  int64  
 3   com_reg_ver_win_rate  29850 non-null  float64
 4   customer_idx          29850 non-null  int64  
 5   enterprise            29850 non-null  int64  
 6   customer_job          29850 non-null  int64  
 7   inquiry_type          29850 non-null  int64  
 8   product_category      29850 non-null  int64  
 9   customer_position     29850 non-null  int64  
 10  response_corporate    29850 non-null  int64  
 11  ver_cus               29850 non-null  int64  
 12  ver_pro               29850 non-null  int64  
 13  ver_win_rate_x        29850 non-null  float64
 14  ver_win_ratio_per_bu  29850 non-null  float64
 15  business_area         29

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

Model Training

In [10]:
#optuna

def objective(trial):
    # Hyperparameter search spaces
    lr_C = trial.suggest_float('lr_C', 1e-10, 1e10, log=True)
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 10, 1000)
    rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32, log=True)
    rf_max_features = trial.suggest_categorical('rf_max_features', ['sqrt', 'log2'])
    gbc_n_estimators = trial.suggest_int('gbc_n_estimators', 10, 1000)
    gbc_learning_rate = trial.suggest_float('gbc_learning_rate', 0.01, 1, log=True)
    gbc_subsample = trial.suggest_float('gbc_subsample', 0.1, 1.0)
    
    xgb_n_estimators = trial.suggest_int('xgb_n_estimators', 10, 1000)
    xgb_max_depth = trial.suggest_int('xgb_max_depth', 2, 10)
    xgb_learning_rate = trial.suggest_float('xgb_learning_rate', 0.01, 1, log=True)
    xgb_min_child_weight = trial.suggest_int('xgb_min_child_weight', 1, 6)
    xgb_subsample = trial.suggest_float('xgb_subsample', 0.1, 1.0)
    xgb_colsample_bytree = trial.suggest_float('xgb_colsample_bytree', 0.1, 1.0)
    
    # CatBoost hyperparameters
    cat_learning_rate = trial.suggest_float('cat_learning_rate', 0.01, 0.3)
    cat_depth = trial.suggest_int('cat_depth', 4, 10)
    cat_l2_leaf_reg = trial.suggest_int('cat_l2_leaf_reg', 1, 10)
    
    # Define the ensemble model
    clf1 = LogisticRegression(C=lr_C, random_state=42, max_iter=2000)
    clf2 = RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth, max_features=rf_max_features, random_state=42)
    clf3 = GradientBoostingClassifier(n_estimators=gbc_n_estimators, learning_rate=gbc_learning_rate, subsample=gbc_subsample, random_state=42)
    clf4 = xgb.XGBClassifier(n_estimators=xgb_n_estimators, max_depth=xgb_max_depth, learning_rate=xgb_learning_rate, min_child_weight=xgb_min_child_weight, subsample=xgb_subsample, colsample_bytree=xgb_colsample_bytree, use_label_encoder=False, eval_metric='logloss')
    clf5 = CatBoostClassifier(learning_rate=cat_learning_rate, depth=cat_depth, l2_leaf_reg=cat_l2_leaf_reg, verbose=0)
    
    
    
    # Assign weights based on hypothetical performance (e.g., 1 for LR, 2 for RF, 3 for GBC)
    ensemble_clf = VotingClassifier(estimators=[
    ('lr', clf1),
    ('rf', clf2),
    ('gbc', clf3),
    ('xgb', clf4),
    ('cat', clf5)],  # Here we add CatBoost                                
    voting='soft', weights=[0.1, 0.1, 0.1, 1.2, 1.0])
    
    # Train the ensemble model
    ensemble_clf.fit(X_train, y_train)
    
     # Predict and evaluate the model
    y_pred = ensemble_clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

In [ ]:
# Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)  # Adjust the number of trials as necessary

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-02-19 06:07:57,714] A new study created in memory with name: no-name-f3a45d80-47e4-46f0-9290-29740edc4b06
[I 2024-02-19 06:08:26,796] Trial 0 finished with value: 0.9556113902847572 and parameters: {'lr_C': 427.9934447715739, 'rf_n_estimators': 697, 'rf_max_depth': 17, 'rf_max_features': 'sqrt', 'gbc_n_estimators': 23, 'gbc_learning_rate': 0.2569036182960593, 'gbc_subsample': 0.6168149598393995, 'xgb_n_estimators': 23, 'xgb_max_depth': 2, 'xgb_learning_rate': 0.13376139799253464, 'xgb_min_child_weight': 6, 'xgb_subsample': 0.9659807014474496, 'xgb_colsample_bytree': 0.9165541686522686, 'cat_learning_rate': 0.1906640185022828, 'cat_depth': 10, 'cat_l2_leaf_reg': 10}. Best is trial 0 with value: 0.9556113902847572.
[I 2024-02-19 06:08:56,252] Trial 1 finished with value: 0.9584589614740369 and parameters: {'lr_C': 115.8030288150785, 'rf_n_estimators': 856, 'rf_max_depth': 19, 'rf_max_features': 'log2', 'gbc_n_estimators': 428, 'gbc_learning_rate': 0.038714121549677176, 'gbc_subsa

In [ ]:
# Define the classifiers with the best hyperparameters
best_params = study.best_trial.params
clf1 = LogisticRegression(C=best_params['lr_C'], random_state=42, max_iter=1000)
clf2 = RandomForestClassifier(n_estimators=best_params['rf_n_estimators'], max_depth=best_params['rf_max_depth'], random_state=42)
clf3 = GradientBoostingClassifier(n_estimators=best_params['gbc_n_estimators'], learning_rate=best_params['gbc_learning_rate'], random_state=42)
clf4 = xgb.XGBClassifier(n_estimators=best_params['xgb_n_estimators'], max_depth=best_params['xgb_max_depth'], learning_rate=best_params['xgb_learning_rate'], min_child_weight=best_params['xgb_min_child_weight'], use_label_encoder=False, eval_metric='logloss')
clf5 = CatBoostClassifier(learning_rate=best_params['cat_learning_rate'], depth=best_params['cat_depth'], l2_leaf_reg=best_params['cat_l2_leaf_reg'], verbose=0)


# Assign weights based on hypothetical performance (e.g., 1 for LR, 2 for RF, 3 for GBC)
ensemble_clf = VotingClassifier(estimators=[
    ('lr', clf1),
    ('rf', clf2),
    ('gbc', clf3),
    ('xgb', clf4),
    ('cat', clf5)],
    voting='soft', weights=[0.1, 0.1, 0.1, 1.2, 1.0])


Modeling

In [ ]:
# Train the ensemble model
ensemble_clf.fit(X_train, y_train)

In [ ]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [ ]:
pred = ensemble_clf.predict(X_test)
get_clf_eval(y_test, pred)